In [1]:
import requests
import json
import os
import climate
from pprint import pprint
import pandas as pd
import numpy as np
import io
import census
from pprint import pprint

In [2]:
ca = census.CensusData()
pgpw = os.getenv("POSTGRES_PASSWORD")
server, engine = ca.connect_to_postgres(password=pgpw)


In [3]:
tmp_df = ca.get_data_for_city("norfolk", years=(2019, 2024))
tmp_df

<!doctype html><html lang="en"><head><title>HTTP Status 404 ? Not Found</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 404 ? Not Found</h1></body></html>


,city,year,male_worker_earnings,female_worker_earnings,pct_below_pov,num_housing_units,num_married_housing_units,avg_household_size,lt_10k_income,gt_10k_lt_15k_income,...,pct_in_retail,pct_in_transportation,pct_in_information,pct_in_finance,pct_in_science,pct_in_education,pct_in_arts,pct_in_pub_ad,state,county
0,"Norfolk city, Virginia",2019,40180,35197,12.7,98476,29431,2.45,8092,4136,...,10.7,5.3,1.5,6.0,12.0,23.2,14.2,6.7,51,710
1,"Norfolk city, Virginia",2021,48211,43074,10.6,102423,32338,2.15,6724,3024,...,11.9,5.3,1.5,5.6,12.5,23.2,11.4,9.2,51,710
2,"Norfolk city, Virginia",2022,50111,41419,13.5,103573,29994,2.28,7384,5165,...,11.0,6.7,1.8,6.8,13.7,22.2,10.6,6.6,51,710
3,"Norfolk city, Virginia",2023,52044,43740,12.0,104467,32049,2.28,6484,5612,...,11.3,6.9,1.0,6.4,11.5,24.7,13.0,6.1,51,710


In [ ]:
ca.upload_city_data_to_postgres(tmp_df, engine=engine)

# CITY YEAR will be the primary key(s)
# all other data will follow after that

In [ ]:
co = climate.ClimateAgent()
station_id = co.get_station_id("Norfolk, Virginia")
climate_df = co.get_data_for_station_id(station_id=station_id, start_date="2022-01-01", end_date="2022-12-31")

In [ ]:
# ca = census.CensusData()

# pprint(ca.access_economic_data("Norfolk", [2022]))

In [ ]:
from html.parser import HTMLParser
#census_url = "https://api.census.gov/data"
year = "2023"
dataset = "acs"
url = f"https://api.census.gov/data/{year}/acs/acs1/profile"


"""
    # https://api.census.gov/data/2022/acs/acs1/profile/variables.html 

    # ones worth using:

    # economic
    DP03_0093E = estimate for earnings for first year male workers 
    DP03_0094E = estimate for earnings for first year female workers
    DP03_0119PE = estimated percentage of people below the poverty line
    DP04_0001E = estimated housing units
    DP02_0002E = estimated number of married housing units
    DP02_0016E = estimated average household size

    DP03_0052E: estimated total households less than $10,000 income
    DP03_0053E: estimated total households between $10-15k
    DP03_0054E: estimated total households between $15-25k
    DP03_0055E: 25-35k
    DP03_0056E: 35-45k
    DP03_0057E: 50-75k
    DP03_0058E: 75-100k
    DP03_0059E: 100-150k
    DP03_0060E: 150-200k
    DP03_0061E: > 200k

    # relationships
    DP02_0026E = men over 15 y/o who have not married
    DP02_0028E = men over 15 y/o who married, but separated
    DP02_0030E = men over 15 y/o who are divorced
    
    # education (currently)
    DP02_0054E = estimated population of kids in nursery school/preschool
    DP02_0055E = estimated population of kids in kindergarten
    DP02_0056E = estimated population of kids in grades 1-8
    DP02_0057E = estimated population of kids in grades 9-12
    DP02_0058E = estimated population of kids in college/grad school

    # education attainment
    DP02_0060PE = estimated population of 25 years and older who have less than 9th grade education
    DP02_0061PE = estimated population with some high school education
    DP02_0062PE = estimated population with high school diploma
    DP02_0063PE = estimated population with some college, no diploma
    DP02_0064PE = estimated population with associates degrees
    DP02_0065PE  = estimated population with bachelor's degree
    DP02_0066PE = estimated population with graduate degrees

    # unemployement rate
    DP03_0009PE

    # industry
    DP03_0033PE: agriculture, forestry, fishing, hunting, mining
    DP03_0034PE: construction
    DP03_0035PE: manufacturing
    DP03_0036PE: wholesale trade
    DP03_0037PE: retail trade
    DP03_0038PE: transportation, warehousing, utilities
    DP03_0039PE: information
    DP03_0040PE: finance, insurance, real estate, rental, leasing
    DP03_0041PE: professional, scientific, administrative, waste management
    DP03_0042PE: educational services/health care/ social assistance
    DP03_0043PE: arts, entertainment, recreation, accommodation, food services
    DP03_0045PE: public administration

"""
variables = "NAME,DP02_0060PE,DP02_0061PE,DP02_0062PE,DP02_0063PE,DP02_0064PE,DP02_0065PE,DP02_0066PE"
print(url)
params = {"key": os.getenv("CENSUS_KEY"),
           "for": "county:*", 
           "get": variables,
           "in": "state:51"
          }
r = requests.get(url, params=params)
values = list(json.loads(r.text))

df = pd.DataFrame(values[1:], columns=values[0])
# df = df.rename(columns={
#     "DP02_0060PE":"<9th",
#     "DP02_0061PE": "some high school",
#     "DP02_0062PE": "high school",
#     "DP02_0063PE": "some college",
#     "DP02_0064PE": "associates",
#     "DP02_0065PE": "bachelors",
#     "DP02_0066PE": "graduate"
# })
df

In [ ]:
df['NAME'] = df['NAME'].str.lower()
df['NAME'] = df['NAME'].str.replace(" ", "")

In [ ]:
df[df['NAME'].str.contains("norfolk")]